## Assignment 4 

The questions and datasets in this assignment were provided by the University of Michigan.  Solutions to the questions are my own.

The assignment involves finding how the population of a city correlates with the performance of said city's sports team. Performance is defined by the team's win-loss ratio, which is the number of wins over the number of wins plus the number of losses.  

This correlation analysis is carried in questions 1 to 4, with each question focusing on one of the 4 major sports leagues in North America - NFL, NHL, NBA, MLB. 

Finally, question 5 addresses the hypothesis that given that a city has two sports teams in different sports, those teams will perform the same within their respective sports. 

### Datasets

4 datasets that each contain sports team performance within a city for the 4 sports leagues - NFL, NHL, NBA, MLB.  Source: University of Michigan

1 dataset for population and sports team names within each city.  Source: Wikipedia

### Question 1 - NHL

Calculate the win/loss ratio's correlation with the population of the city it is in for the NHL using 2018 data.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re


In [2]:
# Import NHL dataset
nhl_df=pd.read_csv("assignments/assignment4/assets/nhl.csv")
nhl_df.head()

# The team column contains the team names. Each team name contains a part of the city name 

,team,GP,W,L,OL,PTS,PTS%,GF,GA,SRS,SOS,RPt%,ROW,year,League
0,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,2018,NHL
1,Tampa Bay Lightning*,82,54,23,5,113,.689,296,236,0.66,-0.07,.634,48,2018,NHL
2,Boston Bruins*,82,50,20,12,112,.683,270,214,0.62,-0.07,.610,47,2018,NHL
3,Toronto Maple Leafs*,82,49,26,7,105,.640,277,232,0.49,-0.06,.567,42,2018,NHL
4,Florida Panthers,82,44,30,8,96,.585,248,246,-0.01,-0.04,.537,41,2018,NHL


In [3]:
# Import city dataset
cities=pd.read_html("assignments/assignment4/assets/wikipedia_data.html")[1]

# Select relevant columns
cities = cities.iloc[:-1,[0,3,5,6,7,8]]

In [4]:
cities.head()

,Metropolitan area,Population (2016 est.)[8],NFL,MLB,NBA,NHL
0,New York City,20153634,GiantsJets[note 1],YankeesMets[note 2],KnicksNets,RangersIslandersDevils[note 3]
1,Los Angeles,13310447,RamsChargers[note 4],DodgersAngels,LakersClippers,KingsDucks
2,San Francisco Bay Area,6657982,49ersRaiders[note 6],GiantsAthletics,Warriors,Sharks[note 7]
3,Chicago,9512999,Bears[note 8],CubsWhite Sox,Bulls[note 9],Blackhawks
4,Dallas–Fort Worth,7233323,Cowboys,Rangers,Mavericks,Stars


The **cities** dataset needs to be merged with the **NHL** dataset to compare the population and team performance within a given city.  At first glance a merge on the Metropolitan area column (**cities**) with team column (**NHL**) seems reasonable.  However, this will miss some teams because of variations in city name.  For example, "Florida Panthers" (**NHL**) would not merge with "Miami–Fort Lauderdale" (**cities**).  It is more accurate to use the NHL column of the **cities** dataset, since this includes the last word of the team name, which should match with the team column (**NHL** dataset).  

Before merging, the team column in **NHL** would have to be split so as to isolate the last word of the team name.  For example,  "Florida Panthers" should be split to "Florida" and "Panthers".  "Panthers" would then match with "Panthers" in NHL column of **cities**.  The isolated name can be placed in a new column titled team2 in **NHL**.

In many cases, the NHL column of **cities** contains multiple team names in a given city.  For example, New York City has 3 teams in the NHL column - "RangersIslandersDevils".  Each capital letter represents the last word of the team name - "Rangers", "Islanders", and "Devils".  This column needs to be split as well, to isolate each team name.  The isolated team names can be placed in a new column in **cities** also titled team2.   

Finally, team2 of **NHL** can merge with team2 of **cities**  

In [5]:
# cities dataset - remove blank spaces in Metropolitan area and NHL columns
cities['Metropolitan area'].str.strip()  # just cleaning this column though we won't be using it for the merge
cities['NHL'].str.strip()

# NHL dataset - remove blank spaces in team names of NHL dataset
nhl_df['team'].str.strip()

0         Atlantic Division
1      Tampa Bay Lightning*
2            Boston Bruins*
3      Toronto Maple Leafs*
4          Florida Panthers
               ...         
166      Los Angeles Kings*
167         Phoenix Coyotes
168       Vancouver Canucks
169          Calgary Flames
170         Edmonton Oilers
Name: team, Length: 171, dtype: object

In [6]:
# Use a regex to split the team name into 2 parts - city name and team name 
pattern_nhl = "(?P<city>.*)(?:\s)(?P<team2>\w*)(?:\**$)"

extract_nhl = nhl_df['team'].str.extract(pattern_nhl)
extract_nhl.head()

,city,team2
0,Atlantic,Division
1,Tampa Bay,Lightning
2,Boston,Bruins
3,Toronto Maple,Leafs
4,Florida,Panthers


In [7]:
# Adding the results of the split to the NHL dataset, which will be used to merge
nhl_df['city'] = extract_nhl['city']
nhl_df['team2'] = extract_nhl['team2']

In [8]:
# Filter for relevant year as per the question
nhl_df = nhl_df[nhl_df['year'] == 2018]

In [9]:
# cities dataset - create a column team2 that will contain last words of team names 
cities['team2'] = cities['NHL'].str.findall('[A-Z][a-z]*')
cities.head()

,Metropolitan area,Population (2016 est.)[8],NFL,MLB,NBA,NHL,team2
0,New York City,20153634,GiantsJets[note 1],YankeesMets[note 2],KnicksNets,RangersIslandersDevils[note 3],"[Rangers, Islanders, Devils]"
1,Los Angeles,13310447,RamsChargers[note 4],DodgersAngels,LakersClippers,KingsDucks,"[Kings, Ducks]"
2,San Francisco Bay Area,6657982,49ersRaiders[note 6],GiantsAthletics,Warriors,Sharks[note 7],[Sharks]
3,Chicago,9512999,Bears[note 8],CubsWhite Sox,Bulls[note 9],Blackhawks,[Blackhawks]
4,Dallas–Fort Worth,7233323,Cowboys,Rangers,Mavericks,Stars,[Stars]


In [10]:
# team2 column contains a list of teams within a city. Individual rows need to be created for each team. 
cities = cities.explode('team2')
cities.dropna(inplace = True)

In [11]:
# Merge NHL and cities datasets on team2
nhl_merge = pd.merge(nhl_df, cities, how='inner', on='team2')

In [12]:
nhl_merge.head()

,team,GP,W,L,OL,PTS,PTS%,GF,GA,SRS,...,year,League,city,team2,Metropolitan area,Population (2016 est.)[8],NFL,MLB,NBA,NHL
0,Tampa Bay Lightning*,82,54,23,5,113,.689,296,236,0.66,...,2018,NHL,Tampa Bay,Lightning,Tampa Bay Area,3032171,Buccaneers,Rays,—,Lightning
1,Boston Bruins*,82,50,20,12,112,.683,270,214,0.62,...,2018,NHL,Boston,Bruins,Boston,4794447,Patriots[note 14],Red Sox[note 15],Celtics,Bruins
2,Toronto Maple Leafs*,82,49,26,7,105,.640,277,232,0.49,...,2018,NHL,Toronto Maple,Leafs,Toronto,5928040,— [note 22],Blue Jays,Raptors[note 23],Maple Leafs
3,Florida Panthers,82,44,30,8,96,.585,248,246,-0.01,...,2018,NHL,Florida,Panthers,Miami–Fort Lauderdale,6066387,Dolphins,Marlins,Heat,Panthers
4,Detroit Red Wings,82,30,39,13,73,.445,217,255,-0.48,...,2018,NHL,Detroit Red,Wings,Detroit,4297617,Lions,Tigers[note 20],Pistons[note 21],Red Wings


In [13]:
# Let's see what wasn't merged
nhl_notmerged = pd.merge(nhl_df, cities, how='left', on='team2', indicator = True)
nhl_notmerged  = nhl_notmerged[nhl_notmerged['_merge'] == 'left_only']
nhl_notmerged

,team,GP,W,L,OL,PTS,PTS%,GF,GA,SRS,...,League,city,team2,Metropolitan area,Population (2016 est.)[8],NFL,MLB,NBA,NHL,_merge
0,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,...,NHL,Atlantic,Division,NaN,NaN,NaN,NaN,NaN,NaN,left_only
9,Metropolitan Division,Metropolitan Division,Metropolitan Division,Metropolitan Division,Metropolitan Division,Metropolitan Division,Metropolitan Division,Metropolitan Division,Metropolitan Division,Metropolitan Division,...,NHL,Metropolitan,Division,NaN,NaN,NaN,NaN,NaN,NaN,left_only
18,Central Division,Central Division,Central Division,Central Division,Central Division,Central Division,Central Division,Central Division,Central Division,Central Division,...,NHL,Central,Division,NaN,NaN,NaN,NaN,NaN,NaN,left_only
26,Pacific Division,Pacific Division,Pacific Division,Pacific Division,Pacific Division,Pacific Division,Pacific Division,Pacific Division,Pacific Division,Pacific Division,...,NHL,Pacific,Division,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [14]:
# Seems that only headers were not merged.  Merge was successful

# Now calculate team performance
# Convert existing data to float
nhl_merge['W'] = nhl_merge['W'].apply(lambda x: float(x))
nhl_merge['L'] = nhl_merge['L'].apply(lambda x: float(x))

# Create new column for team's performance
nhl_merge['W/L Ratio'] = nhl_merge['W'] / (nhl_merge['W'] + nhl_merge['L'])

In [15]:
nhl_merge.head()

,team,GP,W,L,OL,PTS,PTS%,GF,GA,SRS,...,League,city,team2,Metropolitan area,Population (2016 est.)[8],NFL,MLB,NBA,NHL,W/L Ratio
0,Tampa Bay Lightning*,82,54.0,23.0,5,113,.689,296,236,0.66,...,NHL,Tampa Bay,Lightning,Tampa Bay Area,3032171,Buccaneers,Rays,—,Lightning,0.701299
1,Boston Bruins*,82,50.0,20.0,12,112,.683,270,214,0.62,...,NHL,Boston,Bruins,Boston,4794447,Patriots[note 14],Red Sox[note 15],Celtics,Bruins,0.714286
2,Toronto Maple Leafs*,82,49.0,26.0,7,105,.640,277,232,0.49,...,NHL,Toronto Maple,Leafs,Toronto,5928040,— [note 22],Blue Jays,Raptors[note 23],Maple Leafs,0.653333
3,Florida Panthers,82,44.0,30.0,8,96,.585,248,246,-0.01,...,NHL,Florida,Panthers,Miami–Fort Lauderdale,6066387,Dolphins,Marlins,Heat,Panthers,0.594595
4,Detroit Red Wings,82,30.0,39.0,13,73,.445,217,255,-0.48,...,NHL,Detroit Red,Wings,Detroit,4297617,Lions,Tigers[note 20],Pistons[note 21],Red Wings,0.434783


In [16]:
# Since one city can have multuple teams, we want to summarize the results by grouping by city and aggregating on mean value of team performance
nhl_merge = nhl_merge.groupby(['Metropolitan area', 'Population (2016 est.)[8]']).agg({'W/L Ratio':np.mean}) 
nhl_merge.reset_index(inplace=True)

In [17]:
nhl_merge

,Metropolitan area,Population (2016 est.)[8],W/L Ratio
0,Boston,4794447,0.714286
1,Buffalo,1132804,0.357143
2,Calgary,1392609,0.513889
3,Chicago,9512999,0.458333
4,Columbus,2041520,0.600000
5,Dallas–Fort Worth,7233323,0.567568
6,Denver,2853077,0.589041
7,Detroit,4297617,0.434783
8,Edmonton,1321426,0.473684
9,Las Vegas,2155664,0.680000


In [18]:
# To find how city population correlates with team performance, we'll use stat.pearsonr.

# Convert population data to float
nhl_merge['Population (2016 est.)[8]'] = nhl_merge['Population (2016 est.)[8]'].apply(lambda x: float(x))

# Send population and win-loss data to their own lists 
population_by_region = nhl_merge['Population (2016 est.)[8]'].tolist()
win_loss_by_region = nhl_merge['W/L Ratio'].tolist()

In [19]:
# Run the correlation with the lists
result_nhl = stats.pearsonr(population_by_region, win_loss_by_region)
result_nhl

(0.012486162921209923, 0.9497182859911781)

In [20]:
# The first value is the correlation coefficient and the second value is the p-value for non-correlation.  The correlation is weak, at 0.012.
# Select the correlation coefficient only, as per the question requirements.
result_nhl[0]


# A team's performance in the NHL is weakly correlated to its home city's population.  

0.012486162921209923

### Question 2 - NBA

Calculate the win/loss ratio's correlation with the population of the city it is in for the NBA using 2018 data.

The solution to this question is similar to that of question 1.  Comments have been minimized for brevity. 

In [21]:
# Import datasets
nba_df = pd.read_csv("assignments/assignment4/assets/nba.csv")
cities_2 = pd.read_html("assignments/assignment4/assets/wikipedia_data.html")[1]

nba_df.head()

# In this case, the win-loss ratio is already given as W/L%

,team,W,L,W/L%,GB,PS/G,PA/G,SRS,year,League
0,Toronto Raptors* (1),59,23,0.72,—,111.7,103.9,7.29,2018,NBA
1,Boston Celtics* (2),55,27,0.6709999999999999,4.0,104.0,100.4,3.23,2018,NBA
2,Philadelphia 76ers* (3),52,30,0.634,7.0,109.8,105.3,4.3,2018,NBA
3,Cleveland Cavaliers* (4),50,32,0.61,9.0,110.9,109.9,0.59,2018,NBA
4,Indiana Pacers* (5),48,34,0.585,11.0,105.6,104.2,1.18,2018,NBA


In [22]:
# Clean columns

cities_2['Metropolitan area'].str.strip()
cities_2['NBA'].str.strip()
nba_df['team'].str.strip()

0           Toronto Raptors* (1)
1            Boston Celtics* (2)
2        Philadelphia 76ers* (3)
3       Cleveland Cavaliers* (4)
4            Indiana Pacers* (5)
                 ...            
157       San Antonio Spurs* (1)
158         Houston Rockets* (4)
159       Memphis Grizzlies* (7)
160        Dallas Mavericks* (8)
161    New Orleans Pelicans (12)
Name: team, Length: 162, dtype: object

In [23]:
# NBA dataset - Isolate for team name.   

pattern_nba = "(?P<team2>[\w]*)(?:[^\w]*)(?:\(.+\))$"
extract_nba = nba_df['team'].str.extract(pattern_nba)
nba_df['team2'] = extract_nba['team2']

In [24]:
# Filter for 2018
nba_df = nba_df[nba_df['year'] == 2018]

In [25]:
# cities dataset - Isolate for team name
cities_2['team2'] = cities_2['NBA'].str.findall('[A-Z|0-9]{1}[a-z|0-9]*')
cities_2 = cities_2.explode('team2')
cities_2.dropna(inplace = True)

In [26]:
# Merge NBA and cities datasets 
nba_merge = pd.merge(nba_df, cities_2, how='inner', on='team2')

In [27]:
# What wasn't merged
nba_notmerged = pd.merge(nba_df, cities_2, how='left', on='team2', indicator = True)
nba_notmerged  = nba_notmerged[nba_notmerged['_merge'] == 'left_only']
nba_notmerged

# Everything seems to have merged

,team,W,L,W/L%,GB,PS/G,PA/G,SRS,year,League,...,Population (2016 est.)[8],B4,NFL,MLB,NBA,NHL,B6,MLS,CFL,_merge


In [28]:
# Convert W/L% column to numerci type.  Could have used float too. 
nba_merge['W/L%'] = pd.to_numeric(nba_merge['W/L%'])

In [29]:
# Group by city, aggregate on mean W/L ratio
nba_merge = nba_merge.groupby(['Metropolitan area', 'Population (2016 est.)[8]']).agg({'W/L%':np.mean}) 
nba_merge.reset_index(inplace=True)
nba_merge

,Metropolitan area,Population (2016 est.)[8],W/L%
0,Atlanta,5789700,0.2930
1,Boston,4794447,0.6710
2,Charlotte,2474314,0.4390
3,Chicago,9512999,0.3290
4,Cleveland,2055612,0.6100
5,Dallas–Fort Worth,7233323,0.2930
6,Denver,2853077,0.5610
7,Detroit,4297617,0.4760
8,Houston,6772470,0.7930
9,Indianapolis,2004230,0.5850


In [30]:
# Prepare for correlation
nba_merge['Population (2016 est.)[8]'] = nba_merge['Population (2016 est.)[8]'].apply(lambda x: float(x))
population_by_region_nba = nba_merge['Population (2016 est.)[8]'].tolist()
win_loss_by_region_nba = nba_merge['W/L%'].tolist()

In [31]:
result_nba = stats.pearsonr(population_by_region_nba, win_loss_by_region_nba)

In [32]:
result_nba[0]

# A team's performance in the NBA is weakly correlated to its home city's population.  

-0.17636350642182935

### Question 3 - MLB

Calculate the win/loss ratio's correlation with the population of the city it is in for the MLB using 2018 data.

In [33]:
# Import datasets 
mlb_df = pd.read_csv("assignments/assignment4/assets/mlb.csv")
cities_3 = pd.read_html("assignments/assignment4/assets/wikipedia_data.html")[1]
mlb_df.head()

,team,W,L,W-L%,GB,year,League
0,Boston Red Sox,108,54,0.667,--,2018,MLB
1,New York Yankees,100,62,0.617,8.0,2018,MLB
2,Tampa Bay Rays,90,72,0.556,18.0,2018,MLB
3,Toronto Blue Jays,73,89,0.451,35.0,2018,MLB
4,Baltimore Orioles,47,115,0.290,61.0,2018,MLB


In [34]:
cities_3.head()

,Metropolitan area,Country,Pop.rank,Population (2016 est.)[8],B4,NFL,MLB,NBA,NHL,B6,MLS,CFL
0,New York City,United States,1,20153634,9,GiantsJets[note 1],YankeesMets[note 2],KnicksNets,RangersIslandersDevils[note 3],11,Red BullsNew York City FC,—
1,Los Angeles,United States,2,13310447,8,RamsChargers[note 4],DodgersAngels,LakersClippers,KingsDucks,10,GalaxyLos Angeles FC[note 5],—
2,San Francisco Bay Area,United States,6,6657982,6,49ersRaiders[note 6],GiantsAthletics,Warriors,Sharks[note 7],7,Earthquakes,—
3,Chicago,United States,3,9512999,5,Bears[note 8],CubsWhite Sox,Bulls[note 9],Blackhawks,6,Fire,—
4,Dallas–Fort Worth,United States,4,7233323,4,Cowboys,Rangers,Mavericks,Stars,5,FC Dallas,—


In [35]:
# These team names in the MLB dataset, have to be manually changed so that they match with the MLB column in the cities dataset. 

name_change = {"Minnesota Twins" : "Minneapolis Twins",
              "Oakland Athletics" : "San Athletics",
              "Texas Rangers" : "Dallas Rangers",
              "St. Louis Cardinals" : "St Cardinals",
              "Colorado Rockies" : "Denver Rockies",
              "Arizona Diamondbacks" : "Phoenix Diamondbacks"}

mlb_df['team'].replace(to_replace = name_change, inplace = True)

In [36]:
# Cleaning columns

cities_3['Metropolitan area'].str.strip()
cities_3['MLB'].str.strip()
mlb_df['team'].str.strip()

0            Boston Red Sox
1          New York Yankees
2            Tampa Bay Rays
3         Toronto Blue Jays
4         Baltimore Orioles
               ...         
145     Los Angeles Dodgers
146    San Francisco Giants
147        San Diego Padres
148          Denver Rockies
149    Phoenix Diamondbacks
Name: team, Length: 150, dtype: object

In [37]:
# In the case of MLB data, a match is more successful based on the first name of the city and the last word of a team name.
pattern_mlb = "(?P<city_name_first>^[A-za-z]*)(?:\s*\w*\s+)(?P<team_name_last>\w*)"
extract_mlb = mlb_df['team'].str.extract(pattern_mlb)

In [38]:
extract_mlb

,city_name_first,team_name_last
0,Boston,Sox
1,New,Yankees
2,Tampa,Rays
3,Toronto,Jays
4,Baltimore,Orioles
...,...,...
145,Los,Dodgers
146,San,Giants
147,San,Padres
148,Denver,Rockies


In [39]:
mlb_df['city_name_first'] = extract_mlb['city_name_first']
mlb_df['team_name_last'] = extract_mlb['team_name_last']

In [40]:
# Filter for 2018 data
mlb_df = mlb_df[mlb_df['year'] == 2018]

In [41]:
# City dataset - isolate last name of team
cities_3['team_name_last'] = cities_3['MLB'].str.findall('[A-Z|0-9]{1}[a-z|0-9]*')

# City dataset - isolate first name of city
pattern_city = "(?P<city_name_first>^[A-za-z]+)"
cities_3['city_name_first'] = cities_3['Metropolitan area'].str.extract(pattern_city)
cities_3 = cities_3.explode('team_name_last')
cities_3.dropna(inplace = True)

cities_3.head()

,Metropolitan area,Country,Pop.rank,Population (2016 est.)[8],B4,NFL,MLB,NBA,NHL,B6,MLS,CFL,team_name_last,city_name_first
0,New York City,United States,1,20153634,9,GiantsJets[note 1],YankeesMets[note 2],KnicksNets,RangersIslandersDevils[note 3],11,Red BullsNew York City FC,—,Yankees,New
0,New York City,United States,1,20153634,9,GiantsJets[note 1],YankeesMets[note 2],KnicksNets,RangersIslandersDevils[note 3],11,Red BullsNew York City FC,—,Mets,New
0,New York City,United States,1,20153634,9,GiantsJets[note 1],YankeesMets[note 2],KnicksNets,RangersIslandersDevils[note 3],11,Red BullsNew York City FC,—,2,New
1,Los Angeles,United States,2,13310447,8,RamsChargers[note 4],DodgersAngels,LakersClippers,KingsDucks,10,GalaxyLos Angeles FC[note 5],—,Dodgers,Los
1,Los Angeles,United States,2,13310447,8,RamsChargers[note 4],DodgersAngels,LakersClippers,KingsDucks,10,GalaxyLos Angeles FC[note 5],—,Angels,Los


In [42]:
# Merge MLB and cities datasets 
mlb_merge = pd.merge(mlb_df, cities_3, how = 'inner', on = ['city_name_first', 'team_name_last'])

In [43]:
# What wasn't merged
mlb_excluded = pd.merge(mlb_df, cities_3, how = 'left', on = ['city_name_first', 'team_name_last'], indicator = True)
mlb_excluded = mlb_excluded[mlb_excluded['_merge'] == 'left_only']
mlb_excluded

# Nothing left out, merge successful

,team,W,L,W-L%,GB,year,League,city_name_first,team_name_last,Metropolitan area,...,Population (2016 est.)[8],B4,NFL,MLB,NBA,NHL,B6,MLS,CFL,_merge


In [44]:
# Group by city, aggregate on mean Win-loss ratio (labeled W-L% here)
mlb_merge['W-L%'] = mlb_merge['W-L%'].apply(lambda x: float(x))
mlb_merge['Population (2016 est.)[8]'] = mlb_merge['Population (2016 est.)[8]'].apply(lambda x: float(x))
mlb_merge = mlb_merge.groupby(['Metropolitan area', 'Population (2016 est.)[8]']).agg({'W-L%':np.mean}) 
mlb_merge.reset_index(inplace=True)

In [45]:
mlb_merge.head()

,Metropolitan area,Population (2016 est.)[8],W-L%
0,Atlanta,5789700.0,0.556
1,Baltimore,2798886.0,0.290
2,Boston,4794447.0,0.667
3,Chicago,9512999.0,0.483
4,Cincinnati,2165139.0,0.414


In [46]:
# Correlation analysis
population_by_region_mlb = mlb_merge['Population (2016 est.)[8]'].tolist()
win_loss_by_region_mlb = mlb_merge['W-L%'].tolist()
result_mlb = stats.pearsonr(population_by_region_mlb, win_loss_by_region_mlb)

In [47]:
result_mlb[0]

# A team's performance in the MLB is weakly correlated to its home city's population.  

0.15003737475409495

### Question 4 - NFL

Calculate the win/loss ratio's correlation with the population of the city it is in for the NFL using 2018 data.

In [48]:
# Import datasets 

nfl_df = pd.read_csv("assignments/assignment4/assets/nfl.csv")
nfl_df['team'] = nfl_df['team'].str.strip('+ *')

cities_4 = pd.read_html("assignments/assignment4/assets/wikipedia_data.html")[1]

nfl_df.head()

,DSRS,L,League,MoV,OSRS,PA,PD,PF,SRS,SoS,T,W,W-L%,team,year
0,AFC East,AFC East,NFL,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,2018
1,2.1,5,NFL,6.9,3.1,325,111,436,5.2,-1.8,0,11,.688,New England Patriots,2018
2,-5.2,9,NFL,-7.1,-3.6,433,-114,319,-8.8,-1.7,0,7,.438,Miami Dolphins,2018
3,-0.6,10,NFL,-6.6,-6.3,374,-105,269,-6.9,-0.3,0,6,.375,Buffalo Bills,2018
4,-5.9,12,NFL,-6.8,-2.0,441,-108,333,-7.8,-1.1,0,4,.250,New York Jets,2018


In [49]:
cities_4.head()

,Metropolitan area,Country,Pop.rank,Population (2016 est.)[8],B4,NFL,MLB,NBA,NHL,B6,MLS,CFL
0,New York City,United States,1,20153634,9,GiantsJets[note 1],YankeesMets[note 2],KnicksNets,RangersIslandersDevils[note 3],11,Red BullsNew York City FC,—
1,Los Angeles,United States,2,13310447,8,RamsChargers[note 4],DodgersAngels,LakersClippers,KingsDucks,10,GalaxyLos Angeles FC[note 5],—
2,San Francisco Bay Area,United States,6,6657982,6,49ersRaiders[note 6],GiantsAthletics,Warriors,Sharks[note 7],7,Earthquakes,—
3,Chicago,United States,3,9512999,5,Bears[note 8],CubsWhite Sox,Bulls[note 9],Blackhawks,6,Fire,—
4,Dallas–Fort Worth,United States,4,7233323,4,Cowboys,Rangers,Mavericks,Stars,5,FC Dallas,—


In [50]:
# These team names in the NFL dataset, have to be manually changed so that they match with the NFL column in the cities dataset. 

name_change_nfl = {"New England Patriots" : "Boston Patriots",
                  "Tennessee Titans" : "Nashville Titans",
                  "Oakland Raiders" : "San Raiders",
                  "Minnesota Vikings" : "Minneapolis Vikings",
                  "Carolina Panthers" : "Charlotte Panthers",
                  "Arizona Cardinals" : "Phoenix Cardinals"}

nfl_df['team'].replace(to_replace = name_change_nfl, inplace = True)

In [51]:
# Clean data 

cities_4['Metropolitan area'].str.strip()
cities_4['NFL'].str.strip()

0       GiantsJets[note 1]
1     RamsChargers[note 4]
2     49ersRaiders[note 6]
3            Bears[note 8]
4                  Cowboys
5                 Redskins
6                   Eagles
7        Patriots[note 14]
8                  Vikings
9                  Broncos
10                Dolphins
11               Cardinals
12                   Lions
13             — [note 22]
14         Texans[note 24]
15                 Falcons
16              Buccaneers
17                Steelers
18         Browns[note 29]
19                Seahawks
20                 Bengals
21                  Chiefs
22               [note 40]
23         Ravens[note 45]
24                Panthers
25                   Colts
26                  Titans
27               [note 53]
28                  Saints
29          Bills[note 56]
30                       —
31                       —
32                       —
33                       —
34                       —
35                       —
36                       —
3

In [52]:
# In the case of NFL data, a match is more successful based on the first name of the city and the last word of a team name.

pattern_nfl = "(?P<city_name_first>^[A-za-z]*)(?:\s*\w*\s+)(?P<team_name_last>\w*)"
extract_nfl = nfl_df['team'].str.extract(pattern_nfl)

nfl_df['city_name_first'] = extract_nfl['city_name_first']
nfl_df['team_name_last'] = extract_nfl['team_name_last']

In [53]:
# Filter for 2018 data
nfl_df = nfl_df[nfl_df['year'] == 2018]

In [54]:
# City dataset - isolate last name of team
cities_4['team_name_last'] = cities_4['NFL'].str.findall('[A-Z|0-9]{1}[a-z|0-9]*')

# City dataset - isolate first name of city
pattern_city = "(?P<city_name_first>^[A-za-z]+)"
cities_4['city_name_first'] = cities_4['Metropolitan area'].str.extract(pattern_city)
cities_4 = cities_4.explode('team_name_last')
cities_4.dropna(inplace = True)

In [55]:
cities_4.head()

,Metropolitan area,Country,Pop.rank,Population (2016 est.)[8],B4,NFL,MLB,NBA,NHL,B6,MLS,CFL,team_name_last,city_name_first
0,New York City,United States,1,20153634,9,GiantsJets[note 1],YankeesMets[note 2],KnicksNets,RangersIslandersDevils[note 3],11,Red BullsNew York City FC,—,Giants,New
0,New York City,United States,1,20153634,9,GiantsJets[note 1],YankeesMets[note 2],KnicksNets,RangersIslandersDevils[note 3],11,Red BullsNew York City FC,—,Jets,New
0,New York City,United States,1,20153634,9,GiantsJets[note 1],YankeesMets[note 2],KnicksNets,RangersIslandersDevils[note 3],11,Red BullsNew York City FC,—,1,New
1,Los Angeles,United States,2,13310447,8,RamsChargers[note 4],DodgersAngels,LakersClippers,KingsDucks,10,GalaxyLos Angeles FC[note 5],—,Rams,Los
1,Los Angeles,United States,2,13310447,8,RamsChargers[note 4],DodgersAngels,LakersClippers,KingsDucks,10,GalaxyLos Angeles FC[note 5],—,Chargers,Los


In [56]:
# Merge NFL and cities datasets 

nfl_merge = pd.merge(nfl_df, cities_4, how = 'inner', on = ['city_name_first', 'team_name_last'])

In [57]:
# What wasn't merged.
nfl_excluded = pd.merge(nfl_df, cities_4, how = 'left', on = ['city_name_first', 'team_name_last'], indicator = True)
nfl_excluded = nfl_excluded[nfl_excluded['_merge'] == 'left_only']
nfl_excluded.loc[:,:'team']

# Only headers left out.  Merge successful

,DSRS,L,League,MoV,OSRS,PA,PD,PF,SRS,SoS,T,W,W-L%,team
0,AFC East,AFC East,NFL,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East
5,AFC North,AFC North,NFL,AFC North,AFC North,AFC North,AFC North,AFC North,AFC North,AFC North,AFC North,AFC North,AFC North,AFC North
10,AFC South,AFC South,NFL,AFC South,AFC South,AFC South,AFC South,AFC South,AFC South,AFC South,AFC South,AFC South,AFC South,AFC South
15,AFC West,AFC West,NFL,AFC West,AFC West,AFC West,AFC West,AFC West,AFC West,AFC West,AFC West,AFC West,AFC West,AFC West
20,NFC East,NFC East,NFL,NFC East,NFC East,NFC East,NFC East,NFC East,NFC East,NFC East,NFC East,NFC East,NFC East,NFC East
25,NFC North,NFC North,NFL,NFC North,NFC North,NFC North,NFC North,NFC North,NFC North,NFC North,NFC North,NFC North,NFC North,NFC North
30,NFC South,NFC South,NFL,NFC South,NFC South,NFC South,NFC South,NFC South,NFC South,NFC South,NFC South,NFC South,NFC South,NFC South
35,NFC West,NFC West,NFL,NFC West,NFC West,NFC West,NFC West,NFC West,NFC West,NFC West,NFC West,NFC West,NFC West,NFC West


In [58]:
# Group by city and aggregate on mean team Win-Loss ratio (labeled W-L% here)

nfl_merge['W-L%'] = nfl_merge['W-L%'].apply(lambda x: float(x))
nfl_merge['Population (2016 est.)[8]'] = nfl_merge['Population (2016 est.)[8]'].apply(lambda x: float(x))
nfl_merge = nfl_merge.groupby(['Metropolitan area', 'Population (2016 est.)[8]']).agg({'W-L%':np.mean}) 
nfl_merge.reset_index(inplace=True)

In [59]:
# Correlation analysis
population_by_region_nfl = nfl_merge['Population (2016 est.)[8]'].tolist()
win_loss_by_region_nfl = nfl_merge['W-L%'].tolist()
result_nfl = stats.pearsonr(population_by_region_nfl, win_loss_by_region_nfl)

In [60]:
result_nfl[0]

# A team's performance in the NFL is weakly correlated to its home city's population.  

0.004282141436393022

#### Correlation Summary


In [61]:
pd.DataFrame({'NHL' : result_nhl[0], 'MLB' : result_mlb[0], 'NBA' : result_nba[0], 'NFL' : result_nfl[0]}, index = ['Correlation'])

,NHL,MLB,NBA,NFL
Correlation,0.012486,0.150037,-0.176364,0.004282


In all sports leagues, the correlation coefficient is low, which indicates that there is no relation between a city's population and its team's performance.

### Question 5

Assess the following hypothesis: 

given a city with 2 sports teams in 2 different sports leagues, those teams will perform the same within their respective sports.  

In this case, the null hypothesis is: given a city with 2 sports teams in 2 different sports leagues, those teams will NOT perform the same within their respective sports.   

The assignment instructions state that the paired t-test must be used.

In [62]:
# Use the win-loss ratio results from questions 1 to 4.  
# Rename the win-loss column to identify their sports leagues to keep track of the comparison  
nhl_merge = nhl_merge.rename(columns = {"W/L Ratio" : "nhl_ratio"})
nba_merge = nba_merge.rename(columns = {"W/L%" : "nba_ratio"})
mlb_merge = mlb_merge.rename(columns = {"W-L%" : "mlb_ratio"})
nfl_merge = nfl_merge.rename(columns = {"W-L%" : "nfl_ratio"})

In [63]:
# Merge all win-loss results from different sports leagues 
sports_df = pd.merge(nhl_merge, nba_merge, how = 'outer', on = 'Metropolitan area')
sports_df = pd.merge(sports_df, mlb_merge, how = 'outer', on = 'Metropolitan area')
sports_df = pd.merge(sports_df, nfl_merge, how = 'outer', on = 'Metropolitan area')
sports_df.drop(['Population (2016 est.)[8]_x', 'Population (2016 est.)[8]_y'], axis = 1, inplace = True)
sports_df.head()

,Metropolitan area,nhl_ratio,nba_ratio,mlb_ratio,nfl_ratio
0,Boston,0.714286,0.671,0.667,0.688
1,Buffalo,0.357143,NaN,NaN,0.375
2,Calgary,0.513889,NaN,NaN,NaN
3,Chicago,0.458333,0.329,0.483,0.750
4,Columbus,0.600000,NaN,NaN,NaN


In [64]:
# Create empty dataframe for the p-value
p_df = pd.DataFrame()
p_df = p_df.fillna(0)

from itertools import combinations
sports_df = sports_df.set_index('Metropolitan area')
combos = list(combinations(sports_df.columns,2))  # list to compare unique pairs of columns

In [65]:
# initialize empty lists
p_result = []  # result of p-value
p_index = []   # label for sports leagues being compared

for a in combos:
    p_df['col1'] = sports_df[a[0]]
    p_df['col2'] = sports_df[a[1]]
    
    p_df.dropna(inplace = True)
    
    # Hypothesis test p-value
    p_result.append(stats.ttest_rel(p_df['col1'],p_df['col2']))
    
    p_index.append(a[0] + ' vs ' + a[1])
    

In [66]:
p_summary = pd.DataFrame(p_result, p_index)
p_summary['pvalue']

nhl_ratio vs nba_ratio    0.022386
nhl_ratio vs mlb_ratio    0.004545
nhl_ratio vs nfl_ratio    0.100609
nba_ratio vs mlb_ratio    0.819462
nba_ratio vs nfl_ratio    0.914035
mlb_ratio vs nfl_ratio    0.772811
Name: pvalue, dtype: float64

If we choose a confidence level of 95%, then the p-value to accept the alternative hypothesis must be less than 5%.  
This is the case for NHL vs NBA (2.2%), and NHL vs MLB (0.4%), where we must reject the null hypothesis.  This means that the MLB and NBA teams perform the same as their NHL counterparts, for a given city.    